In [1]:
import os
os.chdir('..')

In [2]:
from kernel.type import BoolType
from kernel.term import Var, BoolVars, And, Or, Implies, Not, Eq
from kernel.thm import Thm
from kernel import theory
from kernel import term_ord
from kernel.proofterm import ProofTerm
from logic.conv import rewr_conv, top_conv
from logic.logic import apply_theorem, conj_norm
from logic import basic
from syntax.settings import settings

basic.load_theory('sat')
settings.unicode = True

## Tseitin encoding

DPLL-based SAT solving works with a formula in CNF (conjunctive normal form). That is, the formula is written as a conjunction of clauses, where each clause is a disjunction of literals. To use SAT to decide an arbitrary propositional formula, the first step is to convert into CNF form. One standard way to convert an arbitrary propositional formula to CNF form is using the Tseitin encoding, which we present in this section.

The material in this section is based on Chapter 2 of Handbook of Satisfiability.

The idea of Tseitin encoding is to assign a variable to each subterm of the propositional formula. For variables corresponding to non-atoms, an equality between the variable and a conjunction, disjunction, implication, or equivalence of two variables is constructed, which is then converted to CNF form. For example, in the formula (from the Handbook)

$$ (a \to (c \wedge d)) \vee (b \to (c \wedge e)) $$

variables should be made for each of $a,b,c,d,e$, as well as non-atomic subterms $c\wedge d$, $c\wedge e$, $a\to (c\wedge d)$, $b\to (c\wedge e)$, and the whole formula. In our version, we assign variables also to atomic terms.

The subterms of a logical formula can be computed as follows:

In [3]:
def is_logical(t):
    return t.is_implies() or t.is_equals() or t.is_conj() or t.is_disj() or t.is_not()

def logic_subterms(t):
    def rec(t):
        if not is_logical(t):
            return [t]
        elif t.is_not():
            return rec(t.arg) + [t]
        else:
            return rec(t.arg1) + rec(t.arg) + [t]

    return term_ord.sorted_terms(rec(t))

In [4]:
a, b, c, d, e = BoolVars('a b c d e')
t = Or(Implies(a, And(c, d)), Implies(b, And(c, e)))
print(t)
print(', '.join(str(subt) for subt in logic_subterms(t)))

(a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e)
a, b, c, d, e, c ∧ d, c ∧ e, a ⟶ c ∧ d, b ⟶ c ∧ e, (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e)


Now, we can construct a dictionary that assigns to each term a corresponding variable $x_i$, with $i$ starting from 1:

In [5]:
subterm_dict = dict()
for i, subt in enumerate(logic_subterms(t)):
    subterm_dict[subt] = Var('x' + str(i+1), BoolType)

print('{' + ', '.join('%s: %s' % (k, v) for k, v in subterm_dict.items()) + '}')

{a: x1, b: x2, c: x3, d: x4, e: x5, c ∧ d: x6, c ∧ e: x7, a ⟶ c ∧ d: x8, b ⟶ c ∧ e: x9, (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e): x10}


For each subterm, we get a corresponding equality:

In [6]:
eqs = []
for subt in subterm_dict:
    r = subterm_dict[subt]
    if not is_logical(subt):
        eqs.append(Eq(r, subt))
    elif subt.is_not():
        r1 = subterm_dict[subt.arg]
        eqs.append(Eq(r, Not(r1)))
    else:
        r1 = subterm_dict[subt.arg1]
        r2 = subterm_dict[subt.arg]
        eqs.append(Eq(r, subt.head(r1, r2)))

for eq in eqs:
    print(eq)

x1 ⟷ a
x2 ⟷ b
x3 ⟷ c
x4 ⟷ d
x5 ⟷ e
x6 ⟷ x3 ∧ x4
x7 ⟷ x3 ∧ x5
x8 ⟷ (x1 ⟶ x6)
x9 ⟷ (x2 ⟶ x7)
x10 ⟷ x8 ∨ x9


Each equality can be viewed as introducing a new variable, and so does not affect the satisfiability of the problem. It is also clear that the original term is equivalent to the last introduced variable (in the case $x_{10}$). Hence, after introducing new variables, the original formula is *equisatisfiable* to the formula obtained by conjoining the equalities as well as the last introduced variable. It turns out that for the purpose of SAT solving, we only need to prove one direction of the implication, and equalities to atomic variables are not needed.

The needed theorem can be generated directly:

In [7]:
# Target theorem
concl = [eq for eq in eqs if is_logical(eq.rhs)] + [eqs[-1].lhs]
th = Thm(eqs + [t], And(*concl))
print(th)

x1 ⟷ a, x2 ⟷ b, x3 ⟷ c, x4 ⟷ d, x5 ⟷ e, x10 ⟷ x8 ∨ x9, x6 ⟷ x3 ∧ x4, x7 ⟷ x3 ∧ x5, x8 ⟷ (x1 ⟶ x6), x9 ⟷ (x2 ⟶ x7), (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e) ⊢ (x6 ⟷ x3 ∧ x4) ∧ (x7 ⟷ x3 ∧ x5) ∧ (x8 ⟷ (x1 ⟶ x6)) ∧ (x9 ⟷ (x2 ⟶ x7)) ∧ (x10 ⟷ x8 ∨ x9) ∧ x10


The proof of the theorem can also be generated. The only tricky step is going from the original formula to the last introduced variable by rewriting:

In [8]:
eq_pts = [ProofTerm.assume(eq) for eq in eqs]
encode_pt = ProofTerm.assume(t)
for eq_pt in eq_pts:
    encode_pt = encode_pt.on_prop(top_conv(rewr_conv(eq_pt, sym=True)))
for eq_pt in eq_pts:
    if is_logical(eq_pt.rhs):
        encode_pt = apply_theorem('conjI', eq_pt, encode_pt)
print(encode_pt)

ProofTerm(x1 ⟷ a, x2 ⟷ b, x3 ⟷ c, x4 ⟷ d, x5 ⟷ e, x10 ⟷ x8 ∨ x9, x6 ⟷ x3 ∧ x4, x7 ⟷ x3 ∧ x5, x8 ⟷ (x1 ⟶ x6), x9 ⟷ (x2 ⟶ x7), (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e) ⊢ (x10 ⟷ x8 ∨ x9) ∧ (x9 ⟷ (x2 ⟶ x7)) ∧ (x8 ⟷ (x1 ⟶ x6)) ∧ (x7 ⟷ x3 ∧ x5) ∧ (x6 ⟷ x3 ∧ x4) ∧ x10)


Each equality for non-atomic terms can be expressed in CNF form. Equalities for atomic terms will simply be used for rewriting later. The conversion of each kind of equality to CNF form is expressed in the following theorems:

In [9]:
theory.print_theorem(
    'encode_conj',
    'encode_disj',
    'encode_imp',
    'encode_eq',
    'encode_not'
)

encode_conj: ⊢ (l ⟷ r1 ∧ r2) ⟷ (¬l ∨ r1) ∧ (¬l ∨ r2) ∧ (¬r1 ∨ ¬r2 ∨ l)
encode_disj: ⊢ (l ⟷ r1 ∨ r2) ⟷ (¬l ∨ r1 ∨ r2) ∧ (¬r1 ∨ l) ∧ (¬r2 ∨ l)
encode_imp: ⊢ (l ⟷ (r1 ⟶ r2)) ⟷ (¬l ∨ ¬r1 ∨ r2) ∧ (r1 ∨ l) ∧ (¬r2 ∨ l)
encode_eq: ⊢ (l ⟷ r1 ⟷ r2) ⟷ (¬l ∨ ¬r1 ∨ r2) ∧ (¬l ∨ r1 ∨ ¬r2) ∧ (l ∨ ¬r1 ∨ ¬r2) ∧ (l ∨ r1 ∨ r2)
encode_not: ⊢ (l ⟷ ¬r) ⟷ (l ∨ r) ∧ (¬l ∨ ¬r)


We can apply these rewrite rules directly to `res_pt`:

In [10]:
encode_thms = ['encode_conj', 'encode_disj', 'encode_imp', 'encode_eq', 'encode_not']
for th in encode_thms:
    encode_pt = encode_pt.on_prop(top_conv(rewr_conv(th)))
print(encode_pt)

ProofTerm(x1 ⟷ a, x2 ⟷ b, x3 ⟷ c, x4 ⟷ d, x5 ⟷ e, x10 ⟷ x8 ∨ x9, x6 ⟷ x3 ∧ x4, x7 ⟷ x3 ∧ x5, x8 ⟷ (x1 ⟶ x6), x9 ⟷ (x2 ⟶ x7), (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e) ⊢ ((¬x10 ∨ x8 ∨ x9) ∧ (¬x8 ∨ x10) ∧ (¬x9 ∨ x10)) ∧ ((¬x9 ∨ ¬x2 ∨ x7) ∧ (x2 ∨ x9) ∧ (¬x7 ∨ x9)) ∧ ((¬x8 ∨ ¬x1 ∨ x6) ∧ (x1 ∨ x8) ∧ (¬x6 ∨ x8)) ∧ ((¬x7 ∨ x3) ∧ (¬x7 ∨ x5) ∧ (¬x3 ∨ ¬x5 ∨ x7)) ∧ ((¬x6 ∨ x3) ∧ (¬x6 ∨ x4) ∧ (¬x3 ∨ ¬x4 ∨ x6)) ∧ x10)


Finally, we use the normalization of conjunction described in the previous section (available as `logic.conj_norm`) to simplify the conclusion:

In [11]:
encode_pt = encode_pt.on_prop(conj_norm())
print(encode_pt)

ProofTerm(x1 ⟷ a, x2 ⟷ b, x3 ⟷ c, x4 ⟷ d, x5 ⟷ e, x10 ⟷ x8 ∨ x9, x6 ⟷ x3 ∧ x4, x7 ⟷ x3 ∧ x5, x8 ⟷ (x1 ⟶ x6), x9 ⟷ (x2 ⟶ x7), (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e) ⊢ x10 ∧ (x1 ∨ x8) ∧ (x2 ∨ x9) ∧ (¬x6 ∨ x3) ∧ (¬x6 ∨ x4) ∧ (¬x6 ∨ x8) ∧ (¬x7 ∨ x3) ∧ (¬x7 ∨ x5) ∧ (¬x7 ∨ x9) ∧ (¬x8 ∨ x10) ∧ (¬x9 ∨ x10) ∧ (¬x10 ∨ x8 ∨ x9) ∧ (¬x3 ∨ ¬x4 ∨ x6) ∧ (¬x3 ∨ ¬x5 ∨ x7) ∧ (¬x8 ∨ ¬x1 ∨ x6) ∧ (¬x9 ∨ ¬x2 ∨ x7))


The following is the full code for the main function:

In [12]:
def tseitin_encode(t):
    """Given a propositional formula t, compute its Tseitin encoding.
    
    Returns the proof term for the theorem (eqs, t) |- cnf encoding.
    """

    # Mapping from subterms to newly introduced variables
    subterm_dict = dict()
    for i, subt in enumerate(logic_subterms(t)):
        subterm_dict[subt] = Var('x' + str(i+1), BoolType)

    # Collect list of equations
    eqs = []
    for subt in subterm_dict:
        r = subterm_dict[subt]
        if not is_logical(subt):
            eqs.append(Eq(r, subt))
        elif subt.is_not():
            r1 = subterm_dict[subt.arg]
            eqs.append(Eq(r, Not(r1)))
        else:
            r1 = subterm_dict[subt.arg1]
            r2 = subterm_dict[subt.arg]
            eqs.append(Eq(r, subt.head(r1, r2)))

    # Form the proof term
    eq_pts = [ProofTerm.assume(eq) for eq in eqs]
    encode_pt = ProofTerm.assume(t)
    for eq_pt in eq_pts:
        encode_pt = encode_pt.on_prop(top_conv(rewr_conv(eq_pt, sym=True)))
    for eq_pt in eq_pts:
        if is_logical(eq_pt.rhs):
            encode_pt = apply_theorem('conjI', eq_pt, encode_pt)
    
    # Rewrite using Tseitin rules
    encode_thms = ['encode_conj', 'encode_disj', 'encode_imp', 'encode_eq', 'encode_not']
    for th in encode_thms:
        encode_pt = encode_pt.on_prop(top_conv(rewr_conv(th)))
    
    # Normalize the conjuncts
    return encode_pt.on_prop(conj_norm())

The code is called directly as follows:

In [13]:
a, b, c, d, e = BoolVars('a b c d e')
t = Or(Implies(a, And(c, d)), Implies(b, And(c, e)))
print(tseitin_encode(t))

ProofTerm(x1 ⟷ a, x2 ⟷ b, x3 ⟷ c, x4 ⟷ d, x5 ⟷ e, x10 ⟷ x8 ∨ x9, x6 ⟷ x3 ∧ x4, x7 ⟷ x3 ∧ x5, x8 ⟷ (x1 ⟶ x6), x9 ⟷ (x2 ⟶ x7), (a ⟶ c ∧ d) ∨ (b ⟶ c ∧ e) ⊢ x10 ∧ (x1 ∨ x8) ∧ (x2 ∨ x9) ∧ (¬x6 ∨ x3) ∧ (¬x6 ∨ x4) ∧ (¬x6 ∨ x8) ∧ (¬x7 ∨ x3) ∧ (¬x7 ∨ x5) ∧ (¬x7 ∨ x9) ∧ (¬x8 ∨ x10) ∧ (¬x9 ∨ x10) ∧ (¬x10 ∨ x8 ∨ x9) ∧ (¬x3 ∨ ¬x4 ∨ x6) ∧ (¬x3 ∨ ¬x5 ∨ x7) ∧ (¬x8 ∨ ¬x1 ∨ x6) ∧ (¬x9 ∨ ¬x2 ∨ x7))


We test it on more examples:

In [14]:
# Pelletier's problem 7
print(tseitin_encode(Not(Or(a, Not(Not(Not(a)))))))

ProofTerm(x1 ⟷ a, x2 ⟷ ¬x1, x3 ⟷ ¬x2, x4 ⟷ ¬x3, x6 ⟷ ¬x5, x5 ⟷ x1 ∨ x4, ¬(a ∨ ¬¬¬a) ⊢ x6 ∧ (x2 ∨ x1) ∧ (x3 ∨ x2) ∧ (x4 ∨ x3) ∧ (x6 ∨ x5) ∧ (¬x1 ∨ x5) ∧ (¬x4 ∨ x5) ∧ (¬x2 ∨ ¬x1) ∧ (¬x3 ∨ ¬x2) ∧ (¬x4 ∨ ¬x3) ∧ (¬x6 ∨ ¬x5) ∧ (¬x5 ∨ x1 ∨ x4))


In [15]:
# Pelletier's problem 8, Pierce's law
print(tseitin_encode(Not(Implies(Implies(Implies(a, b), a), a))))

ProofTerm(x1 ⟷ a, x2 ⟷ b, x6 ⟷ ¬x5, x3 ⟷ (x1 ⟶ x2), x4 ⟷ (x3 ⟶ x1), x5 ⟷ (x4 ⟶ x1), ¬(((a ⟶ b) ⟶ a) ⟶ a) ⊢ x6 ∧ (x1 ∨ x3) ∧ (x3 ∨ x4) ∧ (x4 ∨ x5) ∧ (x6 ∨ x5) ∧ (¬x1 ∨ x4) ∧ (¬x1 ∨ x5) ∧ (¬x2 ∨ x3) ∧ (¬x6 ∨ ¬x5) ∧ (¬x3 ∨ ¬x1 ∨ x2) ∧ (¬x4 ∨ ¬x3 ∨ x1) ∧ (¬x5 ∨ ¬x4 ∨ x1))


In [16]:
# Pelletier's problem 12
print(tseitin_encode(Not(Eq(Eq(Eq(a, b), c), Eq(a, Eq(b, c))))))

ProofTerm(x1 ⟷ a, x2 ⟷ b, x3 ⟷ c, x9 ⟷ ¬x8, x4 ⟷ x1 ⟷ x2, x5 ⟷ x2 ⟷ x3, x6 ⟷ x1 ⟷ x5, x7 ⟷ x4 ⟷ x3, x8 ⟷ x7 ⟷ x6, ¬(((a ⟷ b) ⟷ c) ⟷ a ⟷ b ⟷ c) ⊢ x9 ∧ (x9 ∨ x8) ∧ (x4 ∨ x1 ∨ x2) ∧ (x5 ∨ x2 ∨ x3) ∧ (x6 ∨ x1 ∨ x5) ∧ (x7 ∨ x4 ∨ x3) ∧ (x8 ∨ x7 ∨ x6) ∧ (¬x9 ∨ ¬x8) ∧ (x4 ∨ ¬x1 ∨ ¬x2) ∧ (x5 ∨ ¬x2 ∨ ¬x3) ∧ (x6 ∨ ¬x1 ∨ ¬x5) ∧ (x7 ∨ ¬x4 ∨ ¬x3) ∧ (x8 ∨ ¬x7 ∨ ¬x6) ∧ (¬x4 ∨ x1 ∨ ¬x2) ∧ (¬x4 ∨ ¬x1 ∨ x2) ∧ (¬x5 ∨ x2 ∨ ¬x3) ∧ (¬x5 ∨ ¬x2 ∨ x3) ∧ (¬x6 ∨ x1 ∨ ¬x5) ∧ (¬x6 ∨ ¬x1 ∨ x5) ∧ (¬x7 ∨ x4 ∨ ¬x3) ∧ (¬x7 ∨ ¬x4 ∨ x3) ∧ (¬x8 ∨ x7 ∨ ¬x6) ∧ (¬x8 ∨ ¬x7 ∨ x6))
